In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [13]:



df = pd.read_csv('injury-details-2025-q3.csv')


london_boroughs = [
    'Barking & Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley',
    'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich',
    'Hackney', 'Hammersmith & Fulham', 'Haringey', 'Harrow',
    'Havering', 'Hillingdon', 'Hounslow', 'Islington',
    'Kensington & Chelsea', 'Kingston Upon Thames', 'Lambeth',
    'Lewisham', 'Merton', 'Newham', 'Redbridge',
    'Richmond Upon Thames', 'Southwark', 'Sutton',
    'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster',
    'City Of London'
]


df_london = df[df['Borough'].isin(london_boroughs)].copy()



borough_counts = df_london['Borough'].value_counts().reset_index()
borough_counts.columns = ['Borough', 'number_of_incidents']


severe_mask = df_london['Injury outcome'].astype(str).str.contains('taken to hospital', case=False, na=False)
severe_counts = df_london[severe_mask]['Borough'].value_counts().reset_index()
severe_counts.columns = ['Borough', 'number_of_severe_incidents']


result = pd.merge(borough_counts, severe_counts, on='Borough', how='left')
result['number_of_severe_incidents'] = result['number_of_severe_incidents'].fillna(0).astype(int)
result = result.sort_values('number_of_incidents', ascending=False).reset_index(drop=True)
result.to_csv('CSVForQuestion1.csv', index=False)





In [6]:
import geopandas as gpd
import pandas as pd

gdf = gpd.read_file("LondonGeo.json", driver='TopoJSON', layer='boroughs')


gdf['longitude'] = gdf.centroid.x
gdf['latitude'] = gdf.centroid.y


centroids = gdf[['id', 'longitude', 'latitude']].copy()
centroids.columns = ['Borough', 'longitude', 'latitude']


incidents = pd.read_csv("CSVForQuestion1.csv")
merged = incidents.merge(centroids, on='Borough', how='left')
merged.to_csv("CSVForQuestion1WithCentroids.csv", index=False)



c:\Users\mason\AppData\Local\Programs\Python\Python310\lib\site-packages\pyogrio\raw.py:200: RuntimeWarning: driver TopoJSON does not support open option DRIVER
  return ogr_read(
